In [ ]:
!pip install catboost optuna --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving sample_submission.csv to sample_submission.csv
Saving testFeatures.csv to testFeatures.csv
Saving train.csv to train.csv


In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import optuna

# Verileri oku
train = pd.read_csv("train.csv")
test = pd.read_csv("testFeatures.csv")
submission = pd.read_csv("sample_submission.csv")

# Tarih sütunları
train["tarih"] = pd.to_datetime(train["tarih"])
test["tarih"] = pd.to_datetime(test["tarih"])
train["yıl"] = train["tarih"].dt.year
train["ay"] = train["tarih"].dt.month
test["yıl"] = test["tarih"].dt.year
test["ay"] = test["tarih"].dt.month



In [ ]:
print(train.columns.tolist())



['tarih', 'ürün', 'ürün besin değeri', 'ürün kategorisi', 'ürün fiyatı', 'ürün üretim yeri', 'market', 'şehir', 'yıl', 'ay']


In [ ]:
cat_features = ["ürün", "market", "şehir", "ürün kategorisi"]

from sklearn.preprocessing import LabelEncoder

for col in cat_features:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))



In [ ]:

target = "ürün fiyatı"
drop_cols = ["id", "tarih", target]  # id yoksa otomatik geçilir
features = [col for col in train.columns if col not in drop_cols]

X = train[features]
y = train[target]
X_test = test[features]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Tüm object (metinsel) sütunları otomatik bul
object_cols = train.select_dtypes(include=["object"]).columns

for col in object_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))





In [ ]:
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "depth": trial.suggest_int("depth", 4, 10),
        "loss_function": "RMSE",
        "verbose": 0
    }

    model = CatBoostRegressor(**params)
    model.fit(X, y)
    preds = model.predict(X)
    rmse = mean_squared_error(y, preds, squared=False)
    return rmse


In [ ]:
print(train.dtypes[train.dtypes == "object"])


Series([], dtype: object)


In [ ]:
from sklearn.preprocessing import LabelEncoder

object_cols = train.select_dtypes(include=["object"]).columns

for col in object_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))


In [ ]:
target = "ürün fiyatı"
drop_cols = ["id", "tarih", target]  # varsa
features = [col for col in train.columns if col not in drop_cols]

X = train[features]
y = train[target]
X_test = test[features]


In [ ]:
import numpy as np

def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "depth": trial.suggest_int("depth", 4, 10),
        "loss_function": "RMSE",
        "verbose": 0
    }

    model = CatBoostRegressor(**params)
    model.fit(X, y)
    preds = model.predict(X)

    mse = mean_squared_error(y, preds)
    rmse = np.sqrt(mse)  # burası güncellendi
    return rmse



In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)


[I 2025-05-03 21:33:35,328] A new study created in memory with name: no-name-7e9f757f-304b-4ecb-925a-757145ff302c
[I 2025-05-03 21:33:51,273] Trial 0 finished with value: 1.3379015110120158 and parameters: {'iterations': 472, 'learning_rate': 0.17407495477124593, 'depth': 5}. Best is trial 0 with value: 1.3379015110120158.
[I 2025-05-03 21:33:59,193] Trial 1 finished with value: 1.3769234868582174 and parameters: {'iterations': 115, 'learning_rate': 0.12514855279665954, 'depth': 10}. Best is trial 0 with value: 1.3379015110120158.
[I 2025-05-03 21:34:16,354] Trial 2 finished with value: 1.798517295115037 and parameters: {'iterations': 476, 'learning_rate': 0.02862041184668422, 'depth': 7}. Best is trial 0 with value: 1.3379015110120158.
[I 2025-05-03 21:34:41,367] Trial 3 finished with value: 1.0171973603029385 and parameters: {'iterations': 652, 'learning_rate': 0.2243135239759353, 'depth': 7}. Best is trial 3 with value: 1.0171973603029385.
[I 2025-05-03 21:35:17,681] Trial 4 finishe

In [ ]:
best_params = study.best_params
best_params["loss_function"] = "RMSE"
best_params["verbose"] = 0

model = CatBoostRegressor(**best_params)
model.fit(X, y)


In [ ]:
predictions = model.predict(X_test)


In [ ]:
predictions_rounded = np.round(predictions, 2)

final = pd.DataFrame({
    "id": test["id"],
    "ürün fiyatı": predictions_rounded
})

final.to_csv("final_submission.csv", index=False)
print("✅ Tahminler virgülden sonra 2 basamak olacak şekilde final_submission.csv dosyasına yazıldı.")
print(final.head(45504))


✅ Tahminler virgülden sonra 2 basamak olacak şekilde final_submission.csv dosyasına yazıldı.
          id  ürün fiyatı
0          0        84.39
1          1        27.23
2          2        32.89
3          3        19.73
4          4        33.63
...      ...          ...
45499  45499        60.82
45500  45500        68.18
45501  45501        66.65
45502  45502        63.71
45503  45503        77.77

[45504 rows x 2 columns]


In [ ]:
from google.colab import files
files.download("final_submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>